# Transfer Learning with ResNet50 (Keras)

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
num_classes = 10

In [6]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [8]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    return image, label

train_ds = train_ds.map(preprocess).batch(32).prefetch(2)
test_ds = test_ds.map(preprocess).batch(32).prefetch(2)

### Freeze the base

In [9]:
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

### adding new classifier head

In [11]:
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

In [12]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5,
    batch_size=32
)

Epoch 1/5
  96/1563 ━━━━━━━━━━━━━━━━━━━━ 1:02:35 3s/step - accuracy: 0.0910 - loss: 2.4241

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_1/resnet50_1/conv5_block1_2_conv_1/BiasAdd defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_ops.cc:1112
	 [[{{node StatefulPartitionedCall/functional_1/resnet50_1/conv5_block1_2_conv_1/BiasAdd}}]] [Op:__inference_multi_step_on_iterator_11600]

In [ ]:
model.evaluate(x_test, y_test)

## MobileNetV2

In [16]:
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [17]:
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

In [18]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5,
    batch_size=4
)

Epoch 1/5
  70/1563 ━━━━━━━━━━━━━━━━━━━━ 2:17:29 6s/step - accuracy: 0.4754 - loss: 1.4826